# Langchain basics

Using Ollama and Llama3.2 (3b) to run a model locally.

## Chat based models

LLMs running in a chat context, like ChatGPT. Based on message-based input and outputs, for answering human questions and prompts.

In [2]:
from langchain_ollama import ChatOllama

chat = ChatOllama(model="llama3.2")

messages = [
    ("system", "You are a helpful AI assistant."),
    ("human", "What the capital of Canada?"),
]

response = chat.invoke(messages)
print(response.content)

The capital of Canada is Ottawa.


## Embedding models

Transform text into vector representations. Every embedding has the same dimensionality, regardless of input length.

In [1]:
from langchain_ollama import OllamaEmbeddings

model = OllamaEmbeddings(
    model="llama3.2"
)

embeddings = model.embed_documents(
    ["Hello world", "Hello there!", "What's your name?", "My name Jeff", "Hi Jeff!"]
)

print("Number of documents embedded:", len(embeddings))
print("Dimension of each embedding:", len(embeddings[0]))

Number of documents embedded: 5
Dimension of each embedding: 3072


## Vector stores

Traditional DBs are not optimized for high dimensionality data like embeddings. Vector stores are faster and more accurate at storing an retrieving LLM embeddings.

### Retrievers

Interfaces than return documents that match a query. Uses metrics like cosine similarity to compare angle between vectors, which relates to semantic similarity.
- Look up smaller chunks of a document to find related content easier, and then search the whole document for additional context.
- Transform natural queries into logical queries ( ie "What were the revenues in 2021" -> datetime = 2021 ) to quickly filter documents

## Chains

Link together components to perform more complex tasks. For example, populating a prompt template and then feeding it to an LLM.

### LLM Chain

Transform user input into a prompt via a template. The LLM output is formatted via a parser.

In [11]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = "List all the colors of the rainbow"
prompt = PromptTemplate(template=template, input_variables=[])

chat = ChatOllama(model="llama3.2")
chain = prompt | chat | StrOutputParser()

chain.invoke({})

'Here are the colors of the rainbow, in order:\n\n1. Red\n2. Orange\n3. Yellow\n4. Green\n5. Blue\n6. Indigo\n7. Violet'

### Sequential

Chain calls to the LLM, using the outputs as subsequent inputs.

In [14]:
from langchain_core.prompts import PromptTemplate

post_prompt = PromptTemplate.from_template(
    """You are a business owner. Given the theme of a post, write a social media post to share on my socials.

Theme: {theme}
Content: This is social media post based on the theme above:"""
)

review_prompt = PromptTemplate.from_template(
    """You are an expert social media manager. Given the presented social media post, it is your job to write a review for the post.

Social Media Post:
{post}
Review from a Social Media Expert:"""
)

from langchain_ollama import ChatOllama
from langchain.schema import StrOutputParser

llm = ChatOllama(model="llama3.2")

chain = (
    {"post": post_prompt | llm | StrOutputParser()}
    | review_prompt
    | llm
    | StrOutputParser()
)
chain.invoke({"theme": "Having a black friday sale with 50% off on everything."})

'**Review of the Black Friday Social Media Post**\n\nOverall, this social media post is well-structured, eye-catching, and effectively communicates the excitement around the upcoming Black Friday sale. Here\'s a breakdown of its strengths and weaknesses:\n\nStrengths:\n\n1. **Attention-grabbing headline**: The use of "BLACK FRIDAY ALERT!" creates a sense of urgency and grabs the attention of potential customers.\n2. **Clear communication**: The post clearly states the benefits of shopping during the sale (50% off everything) and provides specific details about the sale duration, code to redeem the discount, and a call-to-action to shop now.\n3. **Visual appeal**: The use of emojis (🛍️, 👗, 🏠) adds a touch of playfulness and makes the post more engaging.\n4. **Prominent calls-to-action**: The post includes multiple CTAs, such as "Shop now" and "Use code BLACKFRIDAY50 at checkout," which encourages users to take action.\n\nWeaknesses:\n\n1. **Lack of exclusivity**: While the post creates 

## Memory

Used to maintain context in ongoing dialogues with LLMs. LangChain stores input and output in a structured manner so models can remember and refer to past interactions.